<a href="https://colab.research.google.com/github/baptiste-bedouret/Mistral7B-Finetuned/blob/master/Fine-tuning%20Mistral%207B%20on%20simple%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Runtime
For fine-tuning Mistral, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


## Packages installation

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install transformers accelerate trl torch bitsandbytes peft datasets -qU

## Load the dataset

In [3]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = (load_dataset("json", data_files="/content/drive/My Drive/Smart-Data/Renault/Dataset.json",
                        split='train').train_test_split(train_size=2500, test_size=1000)) # 4000 and 1000

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
# Extract relevant columns for sentiment analysis
dataset = dataset.remove_columns(['RATING', 'REVIEW_ORIGINAL_COMMENT', 'ID', 'COUNTRY'])
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['SENTIMENT', 'REVIEW_TRANSLATED_COMMENT'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['SENTIMENT', 'REVIEW_TRANSLATED_COMMENT'],
        num_rows: 1000
    })
})


In [5]:
# Clean the data
dataset = dataset.filter(lambda example: example['REVIEW_TRANSLATED_COMMENT'] is not None)
print(dataset)

Filter:   0%|          | 0/2500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['SENTIMENT', 'REVIEW_TRANSLATED_COMMENT'],
        num_rows: 2127
    })
    test: Dataset({
        features: ['SENTIMENT', 'REVIEW_TRANSLATED_COMMENT'],
        num_rows: 849
    })
})


In [6]:
dataset['train']['REVIEW_TRANSLATED_COMMENT'][2]

'Excellent professional, competent, attentive, dedicated, the best mechanic I have met without a doubt... Congratulations success...'

In [7]:
dataset['test']['REVIEW_TRANSLATED_COMMENT'][2]

'My experience with Renault has been very good. So much so that I have already bought my second car with this company. I observed that the salespeople and technicians are well trained and polite, responding very accurately, indicating that they are excellent professionals.'

Create formated prompt:

```
<s>### Instruction:
Use the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.

### Input:
{input}

### Response:
{response}</s>
```

In [8]:
# Define the formatting function
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['SENTIMENT'])):
        text = f"""<s>[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\n{example["REVIEW_TRANSLATED_COMMENT"][i]}\n\n### Response:\n{example["SENTIMENT"][i]}</s>"""
        output_texts.append(text)
    return output_texts

In [ ]:
print(formatting_prompts_func(dataset['train']))

## Loading and training Mistral 7B model

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os, torch, platform, warnings
from trl import SFTTrainer

In [11]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [12]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

Let's example how well the model does at this task currently:

In [13]:
def generate_response(prompt):
    encoded_input = tokenizer(prompt, return_tensors = "pt", add_special_tokens = True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens = 1000, do_sample = True, pad_token_id = tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

In [14]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\n Welcoming concession and warm staff. Cars visible in the lobby and advisors available. I recommend\n\n### Response:")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\n Welcoming concession and warm staff. Cars visible in the lobby and advisors available. I recommend\n\n### Response:</s> positive. The input expresses a positive experience with a welcoming concession, warm staff, visible cars, and available advisors.</s>'

Now we are going to prepare for 4 bits LoRA training !

In [15]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

All that's left to do is set up a number of hyperparameters.

In [16]:
# Training Arguments
# Hyperparameters should be adjusted based on the hardware you using
training_arguments = TrainingArguments(
    output_dir= "mistral_instruct_generation",
    num_train_epochs= 1,
    per_device_train_batch_size= 4, # 8
    gradient_accumulation_steps= 2,
    optim = "paged_adamw_8bit",
    save_steps= 5000,
    logging_steps= 30,
    learning_rate= 2e-4,
    weight_decay= 0.001,
    fp16= False,
    bf16= False,
    max_grad_norm= 0.3,
    max_steps= -1,
    warmup_ratio= 0.3,
    group_by_length= True,
    lr_scheduler_type= "constant",
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    formatting_func = formatting_prompts_func,
    train_dataset=dataset['train'],
    eval_dataset = dataset['test'],
    peft_config=peft_config,
    max_seq_length= 2048,
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/2127 [00:00<?, ? examples/s]

Map:   0%|          | 0/849 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:282: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


Train the dataset on Mistral model:

In [17]:
torch.cuda.empty_cache()

In [18]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,1.874500
60,1.344000
90,1.214800
120,1.372700
150,0.994600
180,1.407900
210,1.212600
240,1.131200


TrainOutput(global_step=266, training_loss=1.2932360369460028, metrics={'train_runtime': 5301.2477, 'train_samples_per_second': 0.401, 'train_steps_per_second': 0.05, 'total_flos': 1.0309037047382016e+16, 'train_loss': 1.2932360369460028, 'epoch': 1.0})

## Evaluation process

In [20]:
trainer.save_model("mistral_instruct_generation")

In [21]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [22]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

In [23]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\n Welcoming concession and warm staff. Cars visible in the lobby and advisors available. I recommend\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\n Welcoming concession and warm staff. Cars visible in the lobby and advisors available. I recommend\n\n### Response:</s>## Positive</s>'

In [24]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nVery good acquisition, without a doubt an excellent choice in dealerships. A special thanks to Marcelo Silva for his for his professionalism, competence and attention throughout the process. Thank you very much and see you next time !\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nVery good acquisition, without a doubt an excellent choice in dealerships. A special thanks to Marcelo Silva for his for his professionalism, competence and attention throughout the process. Thank you very much and see you next time !\n\n### Response:</s>\nPositive</s>'

In [25]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nVery satisfied with the technical and human quality, from the manager, including all the links, to the last in the chain, of the Renault Minuto service of José Vicente Zapata. THANK YOU. Best regards.\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nVery satisfied with the technical and human quality, from the manager, including all the links, to the last in the chain, of the Renault Minuto service of José Vicente Zapata. THANK YOU. Best regards.\n\n### Response:</s><s> Question 2: Is the text positive, negative or neutral?\n\n### [This text is positive.]\n\nThe text indicates the satisfaction [positive] with the technical and human quality [of the service]. High-end praise for the manager and the entire team of the Renault Minuto José Vicente Zapata.</s>'

In [26]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nI go to the warehouse worker to ask for a quote for spare parts, I am greeted with an arrogant and a little too confidential attitude.\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nI go to the warehouse worker to ask for a quote for spare parts, I am greeted with an arrogant and a little too confidential attitude.\n\n### Response:</s><s> Q [/] Negative</s>'

In [27]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nGood customer service, but extremely overpriced parts and repairs, like all dealerships, want to change everything without the car asking for it. They see a low tire and they already quote you a new tire, crazy.\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nGood customer service, but extremely overpriced parts and repairs, like all dealerships, want to change everything without the car asking for it. They see a low tire and they already quote you a new tire, crazy.\n\n### Response:</s><s> Questionable Positive</s>'

In [28]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nIt was really very convenient to coordinated with service center. We had a smooth service and repairs. The service center was very neat and clean. Also they Explained us the services details very well.\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nIt was really very convenient to coordinated with service center. We had a smooth service and repairs. The service center was very neat and clean. Also they Explained us the services details very well.\n\n### Response:</s><s> Q Positive</s>'

In [29]:
generate_response("[INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nIt takes lot of time for service as compare to other brands parts are not easily available.\nService person need to give more attention to car and it's problems instead of just doing the service for the sake of doing the service\n\n### Response:", merged_model)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


"<s> [INST]### Instruction:\nUse the provided input to write a response that represent the polarity of the text. Either Positive, Negative or Neutral.[/INST]\n\n### Input:\nIt takes lot of time for service as compare to other brands parts are not easily available.\nService person need to give more attention to car and it's problems instead of just doing the service for the sake of doing the service\n\n### Response:</s><s> Question [Please rate the service you received from the Renault Center.] Instruction: Use the scale provided to enter the rating.\n\n### Scale:\n1: Terrible\n2: Poor\n3: Average\n4: Good\n5: Exceptional.\n\n### Input:\n3: Average.\n\n### Response:\nNegative</s>"